# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,29.98,89,100,3.58,EC,1701462678
1,1,invercargill,-46.4000,168.3500,13.37,60,100,0.99,NZ,1701462871
2,2,dudinka,69.4058,86.1778,-21.88,87,100,5.34,RU,1701462871
3,3,edinburgh of the seven seas,-37.0676,-12.3116,13.53,74,100,7.48,SH,1701462872
4,4,novaya usman',51.6418,39.4086,1.14,89,100,5.65,RU,1701462872


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = city_data_df["Humidity"]*100,
    scale = 0.1,
    color = "City",
    alpha = .5)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                            & (city_data_df["Max Temp"] >= 21)
                            & (city_data_df["Wind Speed"] < 4.50)
                            & (city_data_df["Cloudiness"] < 1)]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,ciudad acuna,29.3232,-100.9522,26.16,26,0,2.57,MX,1701462875
78,78,salalah,17.0151,54.0924,26.05,47,0,2.57,OM,1701462901
150,150,vallenar,-28.5708,-70.7581,25.88,25,0,3.90,CL,1701462927
260,260,dondo,-19.6094,34.7431,25.73,76,0,1.80,MZ,1701462962
305,305,inhambane,-23.8650,35.3833,21.29,66,0,3.98,MZ,1701462980


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,ciudad acuna,MX,29.3232,-100.9522,26,
78,salalah,OM,17.0151,54.0924,47,
150,vallenar,CL,-28.5708,-70.7581,25,
260,dondo,MZ,-19.6094,34.7431,76,
305,inhambane,MZ,-23.8650,35.3833,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories": categories,
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ciudad acuna - nearest hotel: Hotel San Jorge
salalah - nearest hotel: Muscat International Hotel
vallenar - nearest hotel: Hotel del Marqués
dondo - nearest hotel: No hotel found
inhambane - nearest hotel: Hotel de Inhambane
sur - nearest hotel: Sur Hotel
el bauga - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
12,ciudad acuna,MX,29.3232,-100.9522,26,Hotel San Jorge
78,salalah,OM,17.0151,54.0924,47,Muscat International Hotel
150,vallenar,CL,-28.5708,-70.7581,25,Hotel del Marqués
260,dondo,MZ,-19.6094,34.7431,76,No hotel found
305,inhambane,MZ,-23.8650,35.3833,66,Hotel de Inhambane
359,sur,OM,22.5667,59.5289,65,Sur Hotel
389,el bauga,SD,18.2620,33.9081,46,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="EsriUSATopo",
    frame_width = 800, frame_height=500, size="Humidity",
    hover_cols = ["Hotel Name", "Country"])
    

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)